In [17]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

#from importlib import reload  # Python 3.4+

import pickle
import pandas as pd

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']
if 'pygents.token_plot' in sys.modules:
    del sys.modules['pygents.token_plot']


from pygents.token import *
from pygents.text import *
from pygents.util import *
from pygents.plot import plot_bars, plot_dict, matrix_plot
from pygents.token_plot import *

In [18]:
path = 'data/corpora/Russian/'
test_df = pd.read_csv(os.path.join(path,'magicdata/zh_en_ru_100/CORPUS_ZH_EN_RU.txt'),delimiter='\t')
test_texts = list(test_df['ru'])
print(len(test_texts))
test_df[['ru']]

100


,ru
0,Как насчет медицинской страховки? В случае ваш...
1,"Для тех, у кого есть страховка, по договору ст..."
2,Необходимо осознать важность наличия страховки.
3,"На самом деле, это явление действительно очень..."
4,Этому поколению родителей действительно необхо...
...,...
95,Ресо предлагает не только страхование автомоби...
96,"Однако при покупке дома, за исключением первон..."
97,Этот вид финансовых вложений имеет определенны...
98,"Если ваша инвестиционная ориентация верна, то ..."


In [19]:
for text in test_texts:
    print(text)

Как насчет медицинской страховки? В случае вашей семьи, её можно оформить и взрослому и ребенку.
Для тех, у кого есть страховка, по договору страхования они получат компенсацию в размере 300 тысяч рублей.
Необходимо осознать важность наличия страховки.
На самом деле, это явление действительно очень распространено, например, для страхования от несчастных случаев, чем больше вы покупаете, тем больше страхуете.
Этому поколению родителей действительно необходимо покупать страховку.
Ну, прямо сейчас действительно желательно купить страховку.
Машину нужно покупать в полном объеме, а дом можно купить в кредит.
Вы можете купить страховку, страховка, конечно же, делится на множество категорий.
Медицинская страховка очень важна.
Эту часть денег платит страховая компания.
Послушайте, я не знаю, слышали ли вы когда-нибудь об этом, это страховка в Сбере.
Покупка дома на самом деле является инвестицией.
Вы когда-нибудь узнавали об обучении в Альфе?
Если он депонирован в банке, каков результат сложны

In [23]:
del_tokenizer = DelimiterTokenizer()

ru_lex = list(pd.read_csv("https://raw.githubusercontent.com/aigents/aigents-java/master/lexicon_russian.txt",sep='\t',header=None).to_records(index=False))
ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=ru_lex,sortmode=0,cased=True)
ru_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=ru_lex,sortmode=1,cased=True)
ru_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=ru_lex,sortmode=2,cased=True)
print(ru_lex0_tokenizer.count_params())

ru_lex0_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])
ru_lex1_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])
ru_lex2_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])
print(ru_lex0_tokenizer.count_params())


91396
91430


In [24]:
print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex0_tokenizer,debug=False))#sort by len
print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex1_tokenizer,debug=False))#sort by freq
print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex2_tokenizer,debug=False))#sort by len and freq


0.94
0.56
0.91


In [31]:
ru_lex = list(pd.read_csv("https://raw.githubusercontent.com/aigents/aigents-java/master/lexicon_russian.txt",sep='\t',header=None).to_records(index=False))
print(len(ru_lex))
filter_thresholds = [0,0.000001,0.00001,0.0001,0.001,0.01]
for t in filter_thresholds:
    lex = listofpairs_compress_with_loss(ru_lex,t) if t > 0 else ru_lex
    ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=0,cased=True)
    ru_lex1_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=1,cased=True)
    ru_lex2_tokenizer = LexiconIndexedTokenizer(lexicon=lex,sortmode=2,cased=True)
    print(t,ru_lex0_tokenizer.count_params())
    ru_lex0_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])
    ru_lex1_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])
    ru_lex2_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex0_tokenizer,debug=False))#sort by len
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex1_tokenizer,debug=False))#sort by freq
    print(evaluate_tokenizer_f1(test_texts,del_tokenizer,ru_lex2_tokenizer,debug=False))#sort by len and freq
    print()


100000
0 91396
0.94
0.56
0.91

1e-06 91396
0.94
0.56
0.91

1e-05 91396
0.94
0.56
0.91

0.0001 46290
0.89
0.57
0.87

0.001 5391
0.72
0.56
0.71

0.01 399
0.65
0.6
0.65



In [39]:
ru_lex0_tokenizer = LexiconIndexedTokenizer(lexicon=ru_lex,sortmode=0,cased=True)
ru_lex0_tokenizer.extend([(i, ru_lex0_tokenizer.top_weight) for i in list(delimiters)])

In [41]:
for text in test_texts:
    expected = del_tokenizer.tokenize(text)
    actual = ru_lex0_tokenizer.tokenize(text)
    f1 = calc_f1(expected,actual)
    if f1 < 1:
        print(expected)
        print(actual)
        print(round(f1,2))


['На', ' ', 'самом', ' ', 'деле', ',', ' ', 'это', ' ', 'явление', ' ', 'действительно', ' ', 'очень', ' ', 'распространено', ',', ' ', 'например', ',', ' ', 'для', ' ', 'страхования', ' ', 'от', ' ', 'несчастных', ' ', 'случаев', ',', ' ', 'чем', ' ', 'больше', ' ', 'вы', ' ', 'покупаете', ',', ' ', 'тем', ' ', 'больше', ' ', 'страхуете', '.']
['На', ' ', 'самом', ' ', 'деле', ',', ' ', 'это', ' ', 'явление', ' ', 'действительно', ' ', 'очень', ' ', 'распространено', ',', ' ', 'например', ',', ' ', 'для', ' ', 'страхования', ' ', 'от', ' ', 'несчастных', ' ', 'случаев', ',', ' ', 'чем', ' ', 'больше', ' ', 'вы', ' ', 'покупаете', ',', ' ', 'тем', ' ', 'больше', ' ', 'страху', 'е', 'те', '.']
0.96
['Послушайте', ',', ' ', 'я', ' ', 'не', ' ', 'знаю', ',', ' ', 'слышали', ' ', 'ли', ' ', 'вы', ' ', 'когда-нибудь', ' ', 'об', ' ', 'этом', ',', ' ', 'это', ' ', 'страховка', ' ', 'в', ' ', 'Сбере', '.']
['Послушайте', ',', ' ', 'я', ' ', 'не', ' ', 'знаю', ',', ' ', 'слышали', ' ', 'ли', '

['Тот', ',', ' ', 'который', ' ', 'напрямую', ' ', 'привязан', ' ', 'к', ' ', 'банковской', ' ', 'карте', ',', ' ', 'может', ' ', 'использоваться', ' ', 'как', ' ', 'кредитная', ',', ' ', 'так', ' ', 'и', ' ', 'дебетовая', ' ', 'карта', '.']
['Тот', ',', ' ', 'который', ' ', 'напрямую', ' ', 'привязан', ' ', 'к', ' ', 'банковской', ' ', 'карте', ',', ' ', 'может', ' ', 'использоваться', ' ', 'как', ' ', 'кредит', 'на', 'я', ',', ' ', 'так', ' ', 'и', ' ', 'де', 'бе', 'тов', 'ая', ' ', 'карта', '.']
0.87
['Когда', ' ', 'вы', ' ', 'оформляете', ' ', 'страховку', ',', ' ', 'у', ' ', 'вас', ' ', 'не', ' ', 'должно', ' ', 'быть', ' ', 'этих', ' ', 'инфекционных', ' ', 'заболеваний', '.']
['Когда', ' ', 'вы', ' ', 'оф', 'ор', 'мл', 'яе', 'те', ' ', 'страховку', ',', ' ', 'у', ' ', 'вас', ' ', 'не', ' ', 'должно', ' ', 'быть', ' ', 'этих', ' ', 'инфекционных', ' ', 'заболеваний', '.']
0.89
['С', ' ', 'юридической', ' ', 'точки', ' ', 'зрения', ' ', 'страхование', ' ', 'является', ' ', 'еще', 